# Atlas CV Rebounding Tutorial

This notebook shows how to use the `atlas_cv_rebounding` package to:

1. Inspect the Kaggle ShotQuality Rebounding data
2. Train a basic offensive rebound model
3. Run the end-to-end pipeline for a single game (once calibration files are ready)


In [ ]:
import pandas as pd
from pathlib import Path

from atlas_cv_rebounding.config import load_config
from atlas_cv_rebounding.feature_engineering import CourtGeometry, FeatureEngineer
from atlas_cv_rebounding.oreb_model import OrebModel

project_root = Path('..').resolve()
cfg = load_config(project_root / 'config.yaml')

kaggle_pbp_path = project_root / cfg.kaggle_train_pbp
kaggle_locs_path = project_root / cfg.kaggle_train_locs

kaggle_pbp = pd.read_csv(kaggle_pbp_path)
kaggle_locs = pd.read_csv(kaggle_locs_path)

print(kaggle_pbp.head())
print(kaggle_locs.head())


In [ ]:
# Build spatial features for Kaggle plays and train a model

geom = CourtGeometry(
    hoop_x=cfg.hoop_x,
    hoop_y=cfg.hoop_y,
    restricted_radius=cfg.restricted_radius,
    paint_x_min=cfg.paint_x_min,
    paint_x_max=cfg.paint_x_max,
    paint_y_min=cfg.paint_y_min,
    paint_y_max=cfg.paint_y_max,
)

fe = FeatureEngineer(geom)
kaggle_features = fe.build_feature_table(kaggle_locs)
kaggle_full = kaggle_features.merge(kaggle_pbp[['id', 'is_oreb']], on='id')

X = kaggle_full.drop(columns=['id', 'is_oreb'])
y = kaggle_full['is_oreb']

model = OrebModel.create_default()
val_logloss, val_brier = model.fit(X, y)
print(f'Validation logloss: {val_logloss:.4f}, Brier: {val_brier:.4f}')


## Running the full game pipeline

Once you have prepared the calibration CSVs (`homography_points.csv`, `frame_anchors.csv`, `team_map.csv`) and edited `config.yaml` to point at your game video + PBP, you can run the full pipeline directly from this notebook (equivalent to `python run_pipeline.py`).

In [ ]:
from atlas_cv_rebounding.pipeline import run_full_pipeline

# WARNING: this can be slow for a full game. Start with a small test clip.
run_full_pipeline(project_root, project_root / 'config.yaml')
